### Restart and Run All

In [2]:
import calendar
import os
import pandas as pd
import sidetable
import numpy as np
import panel as pn
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from itables import show
pn.extension('tabulator')

# Define a formatting function to format integers with zero decimal places and floats with two decimal places
def format_cell(x):
    if isinstance(x, int):
        return "{:,.0f}".format(x)
    elif isinstance(x, float):
        return "{:.2f}".format(x)
    else:
        return x

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

today = date.today()
print(today)

2025-09-07


### Tables in the process

In [4]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [5]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Weekly
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [6]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

### Create orders from stocks after adjust stocks in port_lite from consensus

In [8]:
sql = text('DELETE FROM orders')
result = conlite.execute(sql)
result.rowcount

16

In [9]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
status_bo_stocks = pd.read_sql(sql, conlite)
status_bo_stocks.shape

(11, 7)

In [10]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
status_is_stocks = pd.read_sql(sql, conlite)
status_is_stocks.shape

(7, 7)

In [11]:
df_orders = pd.concat([status_bo_stocks, status_is_stocks], ignore_index=True)
df_orders.shape

(18, 7)

In [12]:
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,11,11,11,11,11,11
S,7,7,7,7,7,7


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
df_dividend.shape

(30, 2)

In [14]:
df_merge = pd.merge(df_orders, df_dividend, on='name', how='outer', indicator=True)
df_merge.shape

(30, 9)

### If there is no dividend record, must create dividend record before proceed

In [16]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge


### Rerun from here

In [18]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,11,11,11,11,11,11,11
S,7,7,7,7,7,7,7


### Restart and Run All Cells

In [20]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,11,11,11,11,11,11,11
S,7,7,7,7,7,7,7


In [21]:
rcds = df_orders.values.tolist()
len(rcds)

18

In [22]:
for rcd in rcds:
    print(rcd)

['B', 'AIMIRT', 2500, 9.1, 0, 'RND15pct', 'SET', '2025-08-19']
['B', 'CPF', 5000, 24.1, 0, '', 'SET50', '2025-08-29']
['S', 'DIF', 10000, 9.3, 0, 'RD40Pct', 'SET', '2025-08-15']
['B', 'GVREIT', 6000, 6.0, 0, 'RND20Pct', 'SET', '2025-08-27']
['B', 'IVL', 1800, 20.6, 0, 'CSSTGT', 'SET50', '2025-08-27']
['B', 'JMART', 3200, 8.0, 0, 'RD80Pct', 'SET50', '2025-05-09']
['B', 'JMT', 2800, 11.0, 0, 'RD75pct', 'SET50', '2025-08-26']
['S', 'MCS', 6000, 8.75, 0, '20Pct', 'SET', '2025-08-26']
['B', 'NER', 5000, 4.0, 0, 'CSSMIN', 'SET', '2025-08-22']
['B', 'ORI', 10000, 2.0, 0, 'RD70Pct', 'SET100', '2025-03-13']
['S', 'PTG', 3600, 11.4, 0, 'COS', 'SET100', '2025-03-10']
['S', 'PTT', 2500, 34.25, 0, '', 'SET50', '2025-03-06']
['B', 'RCL', 3000, 28.0, 0, 'RD35pct', 'SET100', '2025-08-22']
['S', 'SYNEX', 2500, 13.9, 0, 'RD70pct', 'SET', '2025-08-26']
['B', 'TFFIF', 10000, 6.0, 0, 'RD30Pct', 'SET', '2025-08-28']
['S', 'TVO', 4000, 25.5, 0, 'DTD', 'SET', '2025-08-22']
['B', 'WHAIR', 10000, 4.9, 0, 'RD50p

In [23]:
sql = text("""
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (:trade, :name, :qty, :price, :active, :reason, :market, :xdate)
""")

for rcd in rcds:
    # Convert list to dictionary with column names
    data = {
        'trade': rcd[0],
        'name': rcd[1],
        'qty': rcd[2],
        'price': rcd[3],
        'active': rcd[4],
        'reason': rcd[5],
        'market': rcd[6],
        'xdate': rcd[7]
    }
    
    conlite.execute(sql, data)
    conlite.commit()    

In [24]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders.shape

(18, 9)

In [25]:
file_name = 'orders.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [26]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\orders.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\orders.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\orders.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\orders.csv


In [27]:
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [30]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,CPF,5000,24.10,0,,SET50,2025-08-29
1,5,B,IVL,1800,20.60,0,CSSTGT,SET50,2025-08-27
2,6,B,JMART,3200,8.00,0,RD80Pct,SET50,2025-05-09
3,7,B,JMT,2800,11.00,0,RD75pct,SET50,2025-08-26
4,12,S,PTT,2500,34.25,0,,SET50,2025-03-06


In [31]:
name = 'CPF'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.00000,20.000000,2.000000e+01,20.000000
mean,23.480000,23.82500,23.255000,2.643142e+07,23.510000
std,0.893014,0.84969,0.869649,1.236526e+07,0.828378
min,21.800000,22.30000,21.800000,1.326317e+07,21.900000
25%,22.475000,22.92500,22.200000,1.779897e+07,22.875000
50%,23.850000,24.20000,23.650000,2.405377e+07,23.800000
75%,24.200000,24.50000,23.900000,3.024713e+07,24.050000
max,24.400000,24.70000,24.100000,6.812471e+07,24.500000


### 2) Set100 records

In [33]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,10,B,ORI,10000,2.0,0,RD70Pct,SET100,2025-03-13
1,13,B,RCL,3000,28.0,0,RD35pct,SET100,2025-08-22
2,11,S,PTG,3600,11.4,0,COS,SET100,2025-03-10


In [34]:
name = 'ORI'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,1.917167,1.957833,1.874167,7.573404e+06,1.908167
std,0.245537,0.266204,0.233458,8.047976e+06,0.243516
min,1.530000,1.540000,1.500000,1.470398e+06,1.530000
25%,1.707500,1.730000,1.680000,3.473807e+06,1.707500
50%,1.885000,1.920000,1.850000,5.015402e+06,1.880000
75%,2.150000,2.225000,2.105000,8.188006e+06,2.140000
max,2.420000,2.520000,2.260000,5.145323e+07,2.400000


### 3) Set records

In [36]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AIMIRT,2500,9.10,0,RND15pct,SET,2025-08-19
1,4,B,GVREIT,6000,6.00,0,RND20Pct,SET,2025-08-27
2,9,B,NER,5000,4.00,0,CSSMIN,SET,2025-08-22
3,15,B,TFFIF,10000,6.00,0,RD30Pct,SET,2025-08-28
4,17,B,WHAIR,10000,4.90,0,RD50pct,SET,2025-08-18
5,3,S,DIF,10000,9.30,0,RD40Pct,SET,2025-08-15
6,8,S,MCS,6000,8.75,0,20Pct,SET,2025-08-26
7,14,S,SYNEX,2500,13.90,0,RD70pct,SET,2025-08-26
8,16,S,TVO,4000,25.50,0,DTD,SET,2025-08-22
9,18,S,WHART,5000,9.70,0,RD30Pct,SET,2025-08-18


In [37]:
name = 'DIF'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
formatted_df = df.map(format_cell)

# Display the formatted dataframe
print(formatted_df)

    name        date price  maxp  minp         qty  opnp
0    DIF  2025-09-05  8.55  8.55  8.40  10,904,832  8.45
1    DIF  2025-09-04  8.45  8.50  8.40  11,582,084  8.45
2    DIF  2025-09-03  8.45  8.45  8.25  11,380,456  8.30
3    DIF  2025-09-02  8.25  8.30  8.20   1,698,701  8.25
4    DIF  2025-09-01  8.25  8.30  8.20   5,060,268  8.20
..   ...         ...   ...   ...   ...         ...   ...
115  DIF  2025-03-12  7.80  7.85  7.75   9,655,494  7.85
116  DIF  2025-03-11  7.85  7.85  7.80   8,428,142  7.85
117  DIF  2025-03-10  7.85  7.95  7.80  13,696,119  7.95
118  DIF  2025-03-07  7.95  7.95  7.85   4,403,886  7.90
119  DIF  2025-03-06  7.90  8.00  7.90   5,500,160  8.00

[120 rows x 7 columns]


In [38]:
const.close()
conmy.close()
conlite.commit()
conlite.close()

In [39]:
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-09-07 22:03:33
